In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests

import time
import datetime
import os
import shutil
import json

In [6]:
C='音楽 or アルバム or 新曲　or ノミネート'
officialName=["suchmos","サカナクション","北島三郎","米津玄師","あいみょん"]

In [7]:
chopt=webdriver.ChromeOptions()
appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account":""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}

prefs = {'printing.print_preview_sticky_settings.appState': 
json.dumps(appState)}
chopt.add_experimental_option('prefs', prefs)
chopt.add_argument('--kiosk-printing')
driver = webdriver.Chrome(executable_path='./chromedriver',options=chopt)

driver.get('http://t21.nikkei.co.jp/')

In [8]:
selector= '#frmLogin2 > div > span.loginText.mt10 > label > input'
element = driver.find_element_by_css_selector(selector)
element.send_keys('帳號') #放入個人帳號


selector= '#frmLogin2 > div > span:nth-child(2) > label > input'
element= driver.find_element_by_css_selector(selector)
element.send_keys('密碼') #放入個人密碼

time.sleep(1)

element.send_keys(Keys.ENTER)

time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")
a=soup.find_all('input',attrs={'type':'image'})

try:
    if a[0]['alt']=='再ログイン':
        selector= '#frmLogin1 > div > center > input'
        element= driver.find_element_by_css_selector(selector)
        element.click()
except IndexError:
    print("done")

In [9]:
###按下 [記事検索]
selector= '#nk-mainmenu > div > div:nth-child(1) > div > ul > li.nk-menu-item.nk-menu-contract1.nk-article.haschild > p'
element= driver.find_element_by_css_selector(selector)
element.click()

time.sleep(1)

###將預設的搜尋關鍵字刪除
selector= '#contentsPanel > div > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-art-search-con > div.nk-art-key.fixer > input'
element= driver.find_element_by_css_selector(selector)
element.clear()

time.sleep(1)

###輸入上述的第一組關鍵字
selector= '#contentsPanel > div > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-art-search-con > div.nk-art-key.fixer > input'
element= driver.find_element_by_css_selector(selector)
element.send_keys(C)

time.sleep(1)

###執行搜尋
selector= '#contentsPanel > div > form > div.nk-list-search-header > button'
element= driver.find_element_by_css_selector(selector)
element.click()

time.sleep(3)

###按下[絞り込み]
selector= '#contentsPanel > div > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-selectbox-padding-head.nk-art-mode > select > option:nth-child(2)'
element= driver.find_element_by_css_selector(selector)
element.click()


In [ ]:
def nikkei(List):   
    for i in range(len(List)):    

        ###輸入上述的第二組關鍵字
        selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-art-search-con > div.nk-art-key.fixer > input'
        element= driver.find_element_by_css_selector(selector)
        element.send_keys(List[i])


        ###執行搜尋
        selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > button'
        element= driver.find_element_by_css_selector(selector)
        element.click()

        time.sleep(3)

        ###見出しを表示
        html = driver.page_source
        soup = BeautifulSoup(html, "html5lib")
        a=soup.find_all('button',class_='nk-pn-info-listup nk-pn-info-listup-active',attrs={'name':'listUp'})
        try:
            if a[0].text=='':
                selector= '#contentsPanel > div:nth-child(2) > form > div.nk-pn-info-search-result.nk-np-info-search-result-up > button'
                element= driver.find_element_by_css_selector(selector)
                element.click()
                time.sleep(1)
                selector= 'body > div.nk-popup > div.nk-popup-contents > div > div.nk-popup-btn > button.nk-popup-ok'
                element= driver.find_element_by_css_selector(selector)
                element.click()
                time.sleep(1)
                selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > div.nk-list-search-buttons > button'
                element= driver.find_element_by_css_selector(selector)
                element.click()
                time.sleep(3)
                aa='C:\\Users\\user\\Downloads\\下載.pdf'
                bb='C:\\Users\\user\\Downloads\\'+List[i]+'＿日経'+'.pdf'
                os.rename(aa,bb)
                handle_array = driver.window_handles
                driver.switch_to.window(handle_array[0])

        except IndexError:
            print(List[i]+' : '+'検索結果なし')

        ###印刷
        html = driver.page_source
        soup = BeautifulSoup(html, "html5lib")
        a=soup.find_all('div',class_='nk-pn-info-total')
        try:
            if a[0].text=='0件です':
                selector= '#contentsPanel > div:nth-child(2) > form > div.nk-pn-info-search-result.nk-np-info-search-result-up > div.nk-list-search-buttons > button'
                element= driver.find_element_by_css_selector(selector)
                element.click()
                time.sleep(3)
                aa='C:\\Users\\user\\Downloads\\下載.pdf'
                bb='C:\\Users\\user\\Downloads\\'+List[i]+'＿日経'+'.pdf'
                os.rename(aa,bb)         
            if a[0].text!='0件です':
                print(List[i]+' : '+'----------検索結果あり----------')
        except IndexError:
            print('error')


        ###ウィンドウ切り替え
        handle_array = driver.window_handles
        driver.switch_to.window(handle_array[0])


        ###新規
        selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-selectbox-padding-head.nk-art-mode > select > option:nth-child(1)'
        element= driver.find_element_by_css_selector(selector)
        element.click()

        ###事例C
        selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-art-search-con > div.nk-art-key.fixer > input'
        element= driver.find_element_by_css_selector(selector)
        element.send_keys(C)

        ###検索
        selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > button'
        element= driver.find_element_by_css_selector(selector)
        element.click()

        time.sleep(3)

        ###絞り込み
        selector= '#contentsPanel > div:nth-child(2) > form > div.nk-list-search-header > div.nk-list-search.nk-art-list-search > div.nk-selectbox-padding-head.nk-art-mode > select > option:nth-child(2)'
        element= driver.find_element_by_css_selector(selector)
        element.click()

In [ ]:
nikkei(officialName)